In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import root_mean_squared_error, make_scorer
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
import lime
import lime.lime_tabular
from sklearn.inspection import permutation_importance
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from multiprocessing import Pool
import numpy as np
import logging
from sklearn.model_selection import KFold
from skopt import BayesSearchCV
from skopt.space import Real

In [44]:
def calculate_vif(X):
    vif_data = pd.DataFrame()
    vif_data["Feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(
        X.values, i) for i in range(X.shape[1])]
    return vif_data


def vif_selection(X, y, max_vif_for_calculate=100.0, want_max_vif=15.0):
    features = list(X.columns)
    selected_features = []

    correlations_with_y = abs(X.corrwith(y))

    correlation_matrix = X[features].corr().abs()
    mean_correlation = correlation_matrix.mean()
    weighted_correlation = mean_correlation * (1 - correlations_with_y)
    first_feature = weighted_correlation.idxmin()
    selected_features.append(first_feature)
    features.remove(first_feature)

    while len(features) > 0:
        min_score = float('inf')
        feature_to_add = None

        for feature in features:
            temp_features = selected_features + [feature]
            X_temp = X[temp_features]
            vif_data = calculate_vif(X_temp)
            max_vif = vif_data['VIF'].max()

            score = (max_vif / max_vif_for_calculate) / \
                (correlations_with_y[feature] + 0.01)

            print(f"|     | VIF: {max_vif:.4f} | Corr: {
                  correlations_with_y[feature]:.4f} | Score: {score:.4f} | {feature}")

            if score < min_score:
                min_score = score
                feature_to_add = feature

        if max_vif < max_vif_for_calculate:
            selected_features.append(feature_to_add)
            features.remove(feature_to_add)
            print(f"| add | {feature_to_add} (Score: {min_score:.4f}, Correlation with y: {
                  correlations_with_y[feature_to_add]:.4f})")
        else:
            break

    print("|")
    print("|=========================================")
    print("|                  del                   |")

    while True:
        X_current = X[selected_features]
        current_vif = calculate_vif(X_current)
        max_current_vif = current_vif['VIF'].max()

        if max_current_vif <= want_max_vif:
            break

        print(current_vif.sort_values('VIF', ascending=False))

        max_vif_reduction = 0
        feature_to_remove = None

        for feature in selected_features:
            temp_features = [f for f in selected_features if f != feature]
            X_temp = X[temp_features]
            temp_vif = calculate_vif(X_temp)

            current_vif_sum = current_vif['VIF'].sum()
            temp_vif_sum = temp_vif['VIF'].sum()
            vif_reduction = current_vif_sum - temp_vif_sum

            print(f"\n| del | {feature}:")
            print("| new vif |")
            print(temp_vif.sort_values('VIF', ascending=False))
            print(f"total vif reduction: {vif_reduction:.4f}")

            if vif_reduction > max_vif_reduction:
                max_vif_reduction = vif_reduction
                feature_to_remove = feature

        if feature_to_remove:
            selected_features.remove(feature_to_remove)
            print(f"\n| del | {
                  feature_to_remove} (Total VIF reduction: {max_vif_reduction:.4f})")
            print("|=========================================")

    X_selected = X[selected_features]
    final_vif = calculate_vif(X_selected)

    print("\ngood!")

    return X_selected, final_vif


df = pd.read_csv('scaled2.csv')

x = df.drop(columns=['use [kW]'] +
            [col for col in df.columns if 'kw' in col.lower()] + ["use_rolling_mean_3"])
y = df['use [kW]']


numeric_cols = x.select_dtypes(include=['float64', 'int64']).columns
x_numeric = x[numeric_cols]

vif_data = pd.DataFrame()
vif_data["Feature"] = x_numeric.columns
vif_data["VIF"] = [variance_inflation_factor(
    x_numeric.values, i) for i in range(x_numeric.shape[1])]

X_selected, final_vif = vif_selection(
    x_numeric, y, max_vif_for_calculate=100.0, want_max_vif=15.0)

print("")
print(final_vif.sort_values('VIF', ascending=False))

df_target = pd.read_csv('HomeC_TargetEncoding.csv')

exclude_columns = ['use [kW]', 'is_weekday', 'use_rolling_mean_3']
exclude_columns.extend(
    [col for col in df_target.columns if 'kw' in col.lower()])

target_only_columns = [col for col in df_target.columns
                       if col not in df.columns
                       and col not in exclude_columns]

X_final = pd.concat([X_selected, df_target[target_only_columns]], axis=1)

print(target_only_columns)

/Users/ga111o/Documents/dev/kwu-datamining/env/lib/python3.12/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


|     | VIF: 2.4794 | Corr: 0.7028 | Score: 0.0348 | temperature
|     | VIF: 1.8726 | Corr: 0.4078 | Score: 0.0448 | humidity
|     | VIF: 1.7193 | Corr: 0.2537 | Score: 0.0652 | visibility
|     | VIF: 1.6344 | Corr: 0.1884 | Score: 0.0824 | summary
|     | VIF: 2.6115 | Corr: 0.6854 | Score: 0.0376 | apparentTemperature
|     | VIF: 1.6249 | Corr: 0.7032 | Score: 0.0228 | pressure
|     | VIF: 4.0018 | Corr: 0.8333 | Score: 0.0475 | windSpeed
|     | VIF: 1.3082 | Corr: 0.1374 | Score: 0.0887 | cloudCover
|     | VIF: 1.6518 | Corr: 0.2376 | Score: 0.0667 | windBearing
|     | VIF: 3.9829 | Corr: 0.9556 | Score: 0.0412 | precipIntensity
|     | VIF: 2.1699 | Corr: 0.5185 | Score: 0.0411 | dewPoint
|     | VIF: 2.8093 | Corr: 0.8120 | Score: 0.0342 | precipProbability
|     | VIF: 1.5719 | Corr: 0.1966 | Score: 0.0761 | hour
|     | VIF: 1.2275 | Corr: 0.0105 | Score: 0.5989 | day_of_week
|     | VIF: 2.6961 | Corr: 0.7088 | Score: 0.0375 | temp_humidity_interaction
|     | VIF: 6160

In [48]:
if len(X_final) > len(y):
    X_final = X_final.iloc[:-1]
elif len(y) > len(X_final):
    y = y.iloc[:-1]

X_train, X_temp, y_train, y_temp = train_test_split(
    X_final, y, test_size=0.3, random_state=0)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=0)

print(X_train.shape, X_val.shape, X_test.shape)

(352590, 15) (75555, 15) (75555, 15)


In [49]:
print("|============ linear ==============")
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)
print("|")
print("|============ lasso ==============")
lasso_params = {
    'alpha': Real(0.01, 1),
    'max_iter': [100],
    'tol': [0.001]
}
lasso_bayes = BayesSearchCV(
    Lasso(),
    lasso_params,
    n_iter=10,
    cv=KFold(n_splits=5),
    n_jobs=-1,
    random_state=1
)
lasso_bayes.fit(X_train, y_train)
lasso_model = lasso_bayes.best_estimator_
print("| lasso lambda:", lasso_bayes.best_params_['alpha'])
print("|")
print("|============ ridge ==============")
ridge_params = {
    'alpha': Real(0.01, 1),
    'max_iter': [100],
    'tol': [0.001]
}
ridge_bayes = BayesSearchCV(
    Ridge(),
    ridge_params,
    n_iter=10,
    cv=KFold(n_splits=5),
    n_jobs=-1,
    random_state=1
)
ridge_bayes.fit(X_train, y_train)
ridge_model = ridge_bayes.best_estimator_
print("| ridge lambda:", ridge_bayes.best_params_['alpha'])
print("|")
print("|============ elastic ==============")

elastic_params = {
    'alpha': Real(0.01, 1),  # l1 + l2
    'l1_ratio': Real(0.01, 1),
    'max_iter': [1000],
    'tol': [0.001]
}
elastic_bayes = BayesSearchCV(
    ElasticNet(),
    elastic_params,
    n_iter=30,
    cv=KFold(n_splits=5),
    n_jobs=-1,
    random_state=1
)
elastic_bayes.fit(X_train, y_train)
elastic_model = elastic_bayes.best_estimator_

l1_param = elastic_bayes.best_params_[
    'alpha'] * elastic_bayes.best_params_['l1_ratio']
l2_param = elastic_bayes.best_params_[
    'alpha'] * (1 - elastic_bayes.best_params_['l1_ratio'])

print("| elasticnet lambda (L1):", l1_param)
print("| elasticnet lambda (L2):", l2_param)
print("|")

|============ linear ==============
|
|============ lasso ==============


/Users/ga111o/Documents/dev/kwu-datamining/env/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.775e+04, tolerance: 3.455e+02
  model = cd_fast.enet_coordinate_descent(
/Users/ga111o/Documents/dev/kwu-datamining/env/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.085e+04, tolerance: 3.458e+02
  model = cd_fast.enet_coordinate_descent(
/Users/ga111o/Documents/dev/kwu-datamining/env/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, chec

| lasso lambda: 0.1253129846023188
|
|============ ridge ==============


/Users/ga111o/Documents/dev/kwu-datamining/env/lib/python3.12/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=1.04501e-16): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/Users/ga111o/Documents/dev/kwu-datamining/env/lib/python3.12/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=1.04679e-16): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/Users/ga111o/Documents/dev/kwu-datamining/env/lib/python3.12/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=1.04641e-16): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/Users/ga111o/Documents/dev/kwu-datamining/env/lib/python3.12/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=1.05599e-16): result may not be accurate.
  return 

| ridge lambda: 0.1253129846023188
|
|============ elastic ==============
| elasticnet lambda (L1): 0.0001
| elasticnet lambda (L2): 0.0099
|


In [50]:
linear_predicted_value = linear_model.predict(X_test)
lasso_predicted_value = lasso_model.predict(X_test)
ridge_predicted_value = ridge_model.predict(X_test)
elastic_predicted_value = elastic_model.predict(X_test)


n = df.shape[1]
p = X_test.shape[1]

linear_r2 = r2_score(y_test, linear_predicted_value)
linear_adj_r2 = 1 - (1-linear_r2)*(n-1)/(n-p-1)

lasso_r2 = r2_score(y_test, lasso_predicted_value)
lasso_adj_r2 = 1 - (1-lasso_r2)*(n-1)/(n-p-1)

ridge_r2 = r2_score(y_test, ridge_predicted_value)
ridge_adj_r2 = 1 - (1-ridge_r2)*(n-1)/(n-p-1)

elastic_r2 = r2_score(y_test, elastic_predicted_value)
elastic_adj_r2 = 1 - (1-elastic_r2)*(n-1)/(n-p-1)

coefficients1 = pd.DataFrame({
    'Feature': X_train.columns,
    'Coefficient': linear_model.coef_
})
coefficients2 = pd.DataFrame({
    'Feature': X_train.columns,
    'Coefficient': lasso_model.coef_
})
coefficients3 = pd.DataFrame({
    'Feature': X_train.columns,
    'Coefficient': ridge_model.coef_
})
coefficients4 = pd.DataFrame({
    'Feature': X_train.columns,
    'Coefficient': elastic_model.coef_
})
print("============= linear ====================")
print(f"         R² Score: {linear_r2:.4f}")
print(f"Adjusted R² Score: {linear_adj_r2:.4f}")
print("         Intercept:", linear_model.intercept_)
print(coefficients1.sort_values(by='Coefficient', ascending=False))
print("=========================================")
print("\n\n")
print("============= lasso ====================")
print(f"         R² Score: {lasso_r2:.4f}")
print(f"Adjusted R² Score: {lasso_adj_r2:.4f}")
print("         Intercept:", lasso_model.intercept_)
print(coefficients2.sort_values(by='Coefficient', ascending=False))
print("=========================================")
print("\n\n")
print("============= ridge ====================")
print(f"         R² Score: {ridge_r2:.4f}")
print(f"Adjusted R² Score: {ridge_adj_r2:.4f}")
print("         Intercept:", ridge_model.intercept_)
print(coefficients3.sort_values(by='Coefficient', ascending=False))
print("=========================================")
print("\n\n")
print("============= elastic ====================")
print(f"         R² Score: {elastic_r2:.4f}")
print(f"Adjusted R² Score: {elastic_adj_r2:.4f}")
print("         Intercept:", elastic_model.intercept_)
print(coefficients4.sort_values(by='Coefficient', ascending=False))
print("=========================================")
print("\n\n")

============= linear ====================
         R² Score: 0.9904
Adjusted R² Score: 0.9846
         Intercept: 0.593543299251611
                     Feature  Coefficient
2            precipIntensity    69.556975
1          precipProbability     0.628450
3         gen_rolling_mean_3     0.378619
5                 is_weekend     0.142497
8                icon_cloudy     0.014509
9                   icon_fog     0.011987
10    icon_partly-cloudy-day     0.011914
12                 icon_rain     0.009176
6             icon_clear-day     0.006485
7           icon_clear-night     0.005918
0              use_gen_ratio     0.000002
4                  windSpeed    -0.002701
14                 icon_wind    -0.005879
11  icon_partly-cloudy-night    -0.015125
13                 icon_snow    -0.038986



============= lasso ====================
         R² Score: 0.7526
Adjusted R² Score: 0.6041
         Intercept: -1.1635350044200243
                     Feature  Coefficient
4                 